# Max-Cut with QAOA (PennyLane)

Use PennyLane's variational circuits to solve Max-Cut with QAOA and
measure the runtime of the classical optimization loop.


In [ ]:
import time
import pennylane as qml
from pennylane import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


In [ ]:
n_nodes = 6
G = nx.erdos_renyi_graph(n_nodes, 0.5, seed=1)
pos = nx.spring_layout(G, seed=1)
nx.draw(G, pos, with_labels=True)
plt.title("Max-Cut instance (PennyLane)")
plt.show()


In [ ]:
n = len(G.nodes())

coeffs = []
ops = []
for i, j in G.edges():
    coeffs.append(1.0)
    ops.append(qml.PauliZ(i) @ qml.PauliZ(j))

H = qml.Hamiltonian(coeffs, ops)


In [ ]:
dev = qml.device("default.qubit", wires=n)

def cost_layer(gamma):
    for i, j in G.edges():
        qml.IsingZZ(gamma, wires=[i, j])

def mixer_layer(beta):
    for i in G.nodes():
        qml.RX(2 * beta, wires=i)

@qml.qnode(dev)
def qaoa(params):
    gammas, betas = params
    for i in G.nodes():
        qml.Hadamard(wires=i)
    for g, b in zip(gammas, betas):
        cost_layer(g)
        mixer_layer(b)
    return qml.expval(H)


In [ ]:
p = 2
params = 0.01 * np.random.randn(2, p)
opt = qml.GradientDescentOptimizer(stepsize=0.3)

start = time.perf_counter()
steps = 40
for _ in range(steps):
    params = opt.step(qaoa, params)
end = time.perf_counter()

print("Optimal parameters:", params)
print("Final energy:", qaoa(params))
print(f"Runtime ({steps} steps): {end - start:.4f} s")
